# Nested Cross-Validation on Sucrose Infusions

5 subjects : ['E_A1' 'E_A3' 'E_A5' 'E_A7' 'E_A8']   

Reward/Condition Values: ['None' 'Water' 'Sucrose_5' 'Sucrose_15']  

Regions:   
['PFC_delta' 'PFC_theta' 'PFC_alpha' 'PFC_beta' 'PFC_low_gamma'
 'PFC_high_gamma' 'BLA_delta' 'BLA_theta' 'BLA_alpha' 'BLA_beta'
 'BLA_low_gamma' 'BLA_high_gamma' 'NAc_delta' 'NAc_theta' 'NAc_alpha'
 'NAc_beta' 'NAc_low_gamma' 'NAc_high_gamma' 'vHip_delta' 'vHip_theta'
 'vHip_alpha' 'vHip_beta' 'vHip_low_gamma' 'vHip_high_gamma' ]  
 
Features:   
        ['Time(s)' 'PFC_delta' 'PFC_theta' 'PFC_alpha' 'PFC_beta' 'PFC_low_gamma'
 'PFC_high_gamma' 'BLA_delta' 'BLA_theta' 'BLA_alpha' 'BLA_beta'
 'BLA_low_gamma' 'BLA_high_gamma' 'NAc_delta' 'NAc_theta' 'NAc_alpha'
 'NAc_beta' 'NAc_low_gamma' 'NAc_high_gamma' 'vHip_delta' 'vHip_theta'
 'vHip_alpha' 'vHip_beta' 'vHip_low_gamma' 'vHip_high_gamma' 'Reward'
 'MouseId']


Timepoints: { min: 0s, max: 5221.2s }

## TOC:  
* [Load and Prepare Data](#data)  
* [Setup Model](#model)
* [Run Model on Regions](#execute)
* [Results](#results)

In [47]:
# Use when running on google colab
#from google.colab import drive
#drive.mount('/content/drive')

In [48]:
# -- import packages --
import pandas as pd
import numpy as np 
import time
from matplotlib import pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, KFold,  cross_val_score
from numpy import mean,std
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load and Prepare Data <a class="anchor" id="data"></a>

In [49]:
# sucrose infusions csv dataframe, locally set path
path="C:\\Users\\19802\\Documents\\nibl\\\pilot_connectivity\\mouse_data\\Sucrose_infusions_dataframe_Eric.csv" 
data=pd.read_csv(path) # read in as dataframe

In [50]:
# -- print data info -- 
print("\n[INFO] data columns: \n%s \n\n[INFO] target column 'Reward' values: %s \
      \n\n[INFO] mouse IDs: %s \n \
      \n\n[INFO] columns datatypes: \n%s \
      \n\n[INFO] dataframe preview: \n"%(data.columns.values,
                                              data["Reward"].unique(), data['MouseId'].unique(), data.dtypes))

display(data.head(3))



[INFO] data columns: 
['Time(s)' 'PFC_delta' 'PFC_theta' 'PFC_alpha' 'PFC_beta' 'PFC_low_gamma'
 'PFC_high_gamma' 'BLA_delta' 'BLA_theta' 'BLA_alpha' 'BLA_beta'
 'BLA_low_gamma' 'BLA_high_gamma' 'NAc_delta' 'NAc_theta' 'NAc_alpha'
 'NAc_beta' 'NAc_low_gamma' 'NAc_high_gamma' 'vHip_delta' 'vHip_theta'
 'vHip_alpha' 'vHip_beta' 'vHip_low_gamma' 'vHip_high_gamma' 'Reward'
 'MouseId'] 

[INFO] target column 'Reward' values: ['None' 'Water' 'Sucrose_5' 'Sucrose_15']       

[INFO] mouse IDs: ['E_A1' 'E_A3' 'E_A5' 'E_A7' 'E_A8'] 
       

[INFO] columns datatypes: 
Time(s)            float64
PFC_delta          float64
PFC_theta          float64
PFC_alpha          float64
PFC_beta           float64
PFC_low_gamma      float64
PFC_high_gamma     float64
BLA_delta          float64
BLA_theta          float64
BLA_alpha          float64
BLA_beta           float64
BLA_low_gamma      float64
BLA_high_gamma     float64
NAc_delta          float64
NAc_theta          float64
NAc_alpha          float64
N

,Time(s),PFC_delta,PFC_theta,PFC_alpha,PFC_beta,PFC_low_gamma,PFC_high_gamma,BLA_delta,BLA_theta,BLA_alpha,BLA_beta,BLA_low_gamma,BLA_high_gamma,NAc_delta,NAc_theta,NAc_alpha,NAc_beta,NAc_low_gamma,NAc_high_gamma,vHip_delta,vHip_theta,vHip_alpha,vHip_beta,vHip_low_gamma,vHip_high_gamma,Reward,MouseId
0,0.0,0.000003,0.000003,7.761625e-07,0.000002,0.000002,3.155750e-07,0.000003,0.000005,0.000002,0.000004,0.000012,7.465025e-07,0.000043,0.000022,0.000007,0.000008,0.000010,0.000001,0.000047,0.000040,0.000025,0.000012,0.000010,1.251572e-06,None,E_A1
1,0.1,0.000003,0.000003,8.544600e-07,0.000002,0.000002,3.000700e-07,0.000003,0.000004,0.000003,0.000005,0.000012,6.375900e-07,0.000050,0.000030,0.000006,0.000005,0.000010,0.000001,0.000048,0.000047,0.000022,0.000011,0.000011,8.988825e-07,None,E_A1
2,0.2,0.000004,0.000004,8.624575e-07,0.000002,0.000002,2.604275e-07,0.000002,0.000002,0.000002,0.000005,0.000014,6.077150e-07,0.000053,0.000032,0.000006,0.000006,0.000011,0.000001,0.000035,0.000052,0.000018,0.000013,0.000012,1.226840e-06,None,E_A1


#### Dummy Variables on MouseId

In [51]:
data['MouseId'].unique() # View unique mouse ID labels in columns

array(['E_A1', 'E_A3', 'E_A5', 'E_A7', 'E_A8'], dtype=object)

In [52]:
data=pd.get_dummies(data, columns=["MouseId"]) # convert to dummy variables

In [53]:
print("\n[INFO] new dummy data: \n\n")
display(data.head(3))


[INFO] new dummy data: 




,Time(s),PFC_delta,PFC_theta,PFC_alpha,PFC_beta,PFC_low_gamma,PFC_high_gamma,BLA_delta,BLA_theta,BLA_alpha,BLA_beta,BLA_low_gamma,BLA_high_gamma,NAc_delta,NAc_theta,NAc_alpha,NAc_beta,NAc_low_gamma,NAc_high_gamma,vHip_delta,vHip_theta,vHip_alpha,vHip_beta,vHip_low_gamma,vHip_high_gamma,Reward,MouseId_E_A1,MouseId_E_A3,MouseId_E_A5,MouseId_E_A7,MouseId_E_A8
0,0.0,0.000003,0.000003,7.761625e-07,0.000002,0.000002,3.155750e-07,0.000003,0.000005,0.000002,0.000004,0.000012,7.465025e-07,0.000043,0.000022,0.000007,0.000008,0.000010,0.000001,0.000047,0.000040,0.000025,0.000012,0.000010,1.251572e-06,None,1,0,0,0,0
1,0.1,0.000003,0.000003,8.544600e-07,0.000002,0.000002,3.000700e-07,0.000003,0.000004,0.000003,0.000005,0.000012,6.375900e-07,0.000050,0.000030,0.000006,0.000005,0.000010,0.000001,0.000048,0.000047,0.000022,0.000011,0.000011,8.988825e-07,None,1,0,0,0,0
2,0.2,0.000004,0.000004,8.624575e-07,0.000002,0.000002,2.604275e-07,0.000002,0.000002,0.000002,0.000005,0.000014,6.077150e-07,0.000053,0.000032,0.000006,0.000006,0.000011,0.000001,0.000035,0.000052,0.000018,0.000013,0.000012,1.226840e-06,None,1,0,0,0,0


In [54]:
data.columns.values

array(['Time(s)', 'PFC_delta', 'PFC_theta', 'PFC_alpha', 'PFC_beta',
       'PFC_low_gamma', 'PFC_high_gamma', 'BLA_delta', 'BLA_theta',
       'BLA_alpha', 'BLA_beta', 'BLA_low_gamma', 'BLA_high_gamma',
       'NAc_delta', 'NAc_theta', 'NAc_alpha', 'NAc_beta', 'NAc_low_gamma',
       'NAc_high_gamma', 'vHip_delta', 'vHip_theta', 'vHip_alpha',
       'vHip_beta', 'vHip_low_gamma', 'vHip_high_gamma', 'Reward',
       'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7',
       'MouseId_E_A8'], dtype=object)

---

### Prepare Reward Condition Masks

Setup target variables y, by reward

In [55]:
data["Reward"].unique()

array(['None', 'Water', 'Sucrose_5', 'Sucrose_15'], dtype=object)

In [56]:
# -- Setup the target variables  --

Y = data.loc[:, "Reward"] # set copy of target variable to Y 

# make condition masks for Water vs. Sucrose %5 and Water vs. Sucorse vs 15%
s5_cond_mask=Y.isin(['Water', "Sucrose_5"])
s15_cond_mask=Y.isin(['Water', "Sucrose_15"])

# filter data for target variabes w/ condition masks
y_s5=Y[s5_cond_mask]
y_s15=Y[s15_cond_mask]

# prepare session labels for conditions
#session_s5 = data[s5_cond_mask].to_records(index=False)
#session_s15 = data[s15_cond_mask].to_records(index=False)

# prepare feature dataframe with filtered masks
s5_data = data[s5_cond_mask]
s15_data = data[s15_cond_mask]


y_s5.unique(), y_s15.unique()
#session_5.dtype.names, session_15.dtype.names

(array(['Water', 'Sucrose_5'], dtype=object),
 array(['Water', 'Sucrose_15'], dtype=object))

Label Encoding

In [57]:
le = preprocessing.LabelEncoder() # initialize encoder obj
y_s5_enc = le.fit_transform(y_s5) # fit and transform the 5% data
y_s15_enc = le.fit_transform(y_s15) # fit and transform the 15% data

y_s5_enc.shape, y_s15_enc.shape 

((5151,), (5202,))

### Prepare Input Features X by Regions

In [58]:
data.columns.values

array(['Time(s)', 'PFC_delta', 'PFC_theta', 'PFC_alpha', 'PFC_beta',
       'PFC_low_gamma', 'PFC_high_gamma', 'BLA_delta', 'BLA_theta',
       'BLA_alpha', 'BLA_beta', 'BLA_low_gamma', 'BLA_high_gamma',
       'NAc_delta', 'NAc_theta', 'NAc_alpha', 'NAc_beta', 'NAc_low_gamma',
       'NAc_high_gamma', 'vHip_delta', 'vHip_theta', 'vHip_alpha',
       'vHip_beta', 'vHip_low_gamma', 'vHip_high_gamma', 'Reward',
       'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7',
       'MouseId_E_A8'], dtype=object)

Features X input setup

In [60]:
# -- setup delta --
s5_db_data=s5_data[["BLA_delta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_db_data=s15_data[["BLA_delta", 'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s5_dnac_data=s5_data[["NAc_delta", 'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_dnac_data=s15_data[["NAc_delta",'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features

# -- setup beta --
s5_bpfc_data=s5_data[["PFC_beta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_bpfc_data=s15_data[["PFC_beta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s5_bnac_data=s5_data[["NAc_beta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_bnac_data=s15_data[["NAc_beta", 'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s5_bhip_data=s5_data[["vHip_beta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_bhip_data=s15_data[["vHip_beta",  'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features

# -- setup low gamma --
s5_lgnac_data=s5_data[["NAc_low_gamma",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_lgnac_data=s15_data[["NAc_low_gamma", 'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s5_lghip_data=s5_data[["vHip_low_gamma",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_lghip_data=s15_data[["vHip_low_gamma", 'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features


# -- setup theta --
s5_tnac_data=s5_data[["NAc_theta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_tnac_data=s15_data[["NAc_theta", 'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s5_thip_data=s5_data[["vHip_theta",  'MouseId_E_A1', 'MouseId_E_A3', 'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features
s15_thip_data=s15_data[["vHip_theta", 'MouseId_E_A1', 'MouseId_E_A3',  'MouseId_E_A5', 'MouseId_E_A7', 'MouseId_E_A8']] # get target features


**Normalize Data**

In [61]:
# -- scale data -- 

# initialize scaler object
scaler = preprocessing.StandardScaler()

# delta BLA and NAc setup
X_s5_db = scaler.fit_transform(s5_db_data) # fit and transform 
X_s15_db = scaler.fit_transform(s15_db_data) # fit and transform 
X_s5_dnac = scaler.fit_transform(s5_dnac_data) # fit and transform 
X_s15_dnac = scaler.fit_transform(s15_dnac_data) # fit and transform 

# beta PFC and NAc and vHip
X_s5_bpfc = scaler.fit_transform(s5_bpfc_data) # fit and transform 
X_s15_bpfc = scaler.fit_transform(s15_bpfc_data) # fit and transform 
X_s5_bnac = scaler.fit_transform(s5_bnac_data) # fit and transform 
X_s15_bnac = scaler.fit_transform(s15_bnac_data) # fit and transform 
X_s5_bhip = scaler.fit_transform(s5_bhip_data) # fit and transform 
X_s15_bhip = scaler.fit_transform(s15_bhip_data) # fit and transform 

# low gamma NAc and vHip setup
X_s5_lgnac = scaler.fit_transform(s5_lgnac_data) # fit and transform 
X_s15_lgnac = scaler.fit_transform(s15_lgnac_data) # fit and transform 
X_s5_lghip = scaler.fit_transform(s5_lghip_data) # fit and transform 
X_s15_lghip = scaler.fit_transform(s15_lghip_data) # fit and transform 

# -- setup low gamma --
X_s5_tnac = scaler.fit_transform(s5_tnac_data) # fit and transform 
X_s15_tnac = scaler.fit_transform(s15_tnac_data) # fit and transform 
X_s5_thip = scaler.fit_transform(s5_thip_data) # fit and transform 
X_s15_thip = scaler.fit_transform(s15_thip_data) # fit and transform 


X_s5_db.shape, X_s15_db.shape ,X_s5_dnac.shape, X_s15_dnac.shape 

((5151, 6), (5202, 6), (5151, 6), (5202, 6))

## Model Setup: SVM w/ KFold Nested CV  <a class="anchor" id="model"></a>

Set up cross-validation parameters, and SVM 

In [62]:

# configure the cross-validation procedure
cv_inner = KFold(n_splits=10, shuffle=True, random_state=1)
cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)

# define the classifiers
svc_linear = SVC(kernel='linear', max_iter=10000)
svc_rbf = SVC(kernel='rbf', max_iter=10000)

# define the hyperparameter grid
C_range = np.logspace(-3, 2, 6)
gamma_range = np.logspace(-3, 2, 6)

param_grid_linear = dict(C=C_range)
param_grid_rbf = dict(gamma=gamma_range, C=C_range)

# set grid object
grid_linear = GridSearchCV(svc_linear, param_grid_linear, scoring='accuracy', verbose=1, n_jobs=3, cv=cv_inner)
grid_rbf = GridSearchCV(svc_rbf,param_grid_rbf , scoring='accuracy', n_jobs=3, cv=cv_inner)

grid_linear, grid_rbf

(GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
              estimator=SVC(kernel='linear', max_iter=10000), n_jobs=3,
              param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
              scoring='accuracy', verbose=1),
 GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
              estimator=SVC(max_iter=10000), n_jobs=3,
              param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                          'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
              scoring='accuracy'))

## Run Models <a class="anchor" id="execute"></a>

In [82]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_tnac_rbf = cross_val_score(grid_rbf, X_s5_tnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #rbf kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 151.93232202529907 seconds ---


In [83]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tnac_rbf = cross_val_score(grid_rbf, X_s15_tnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #rbf kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 145.74954080581665 seconds ---


In [84]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_tnac_linear = cross_val_score(grid_linear, X_s5_tnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #rbf kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 14.548660039901733 seconds ---


In [85]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_tnac_linear = cross_val_score(grid_linear, X_s15_tnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #rbf kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 14.901972532272339 seconds ---


In [86]:
print('THETA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' %(mean(scores_s5_tnac_linear), std(scores_s5_tnac_linear)))

print('THETA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' %(mean(scores_s5_tnac_rbf), std(scores_s5_tnac_rbf)))

THETA NAc, SVM w/ Linear kernel, Accuracy: 0.520 (0.010)
THETA NAc, SVM w/ RBF kernel, Accuracy: 0.556 (0.005)


In [87]:
print('LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tnac_linear), std(scores_s15_tnac_linear)))

print('LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_tnac_rbf), std(scores_s15_tnac_rbf)))


LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: 0.534 (0.009)
LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: 0.556 (0.012)



>  Delta BLA: Water vs. Sucrose 5%


In [33]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_db_rbf = cross_val_score(grid_rbf, X_s5_db, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #rbf kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 199.25148963928223 seconds ---


In [34]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_db_linear = cross_val_score(grid_linear, X_s5_db, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #linear kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 20.804875135421753 seconds ---


In [35]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_db_rbf = cross_val_score(grid_rbf, X_s15_db, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 187.13819980621338 seconds ---


> Delta BLA: Water vs. Sucrose 15%

In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_db_rbf = cross_val_score(grid_rbf, X_s15_db, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [17]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_db_linear = cross_val_score(grid_linear, X_s15_db, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 122.19924855232239 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_db_poly = cross_val_score(grid_poly, X_s15_db, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

> Delta NAc: Water vs. Sucrose 5%

In [18]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_dnac_linear = cross_val_score(grid_linear, X_s5_dnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 128.1135711669922 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_dnac_rbf = cross_val_score(grid_rbf, X_s5_dnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))



In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_dnac_poly = cross_val_score(grid_poly, X_s5_dnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

 > Delta NAc: Water vs. Sucrose 15%  

In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_dnac_rbf = cross_val_score(grid_rbf, X_s15_dnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [19]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_dnac_linear = cross_val_score(grid_linear, X_s15_dnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 116.70708084106445 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_dnac_poly = cross_val_score(grid_poly, X_s15_dnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

#### Beta

> Beta PFC: Water vs Sucrose %5 

In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bpfc_rbf = cross_val_score(grid_rbf, X_s5_bpfc, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [20]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bpfc_linear = cross_val_score(grid_linear, X_s5_bpfc, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 127.09351086616516 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bpfc_poly = cross_val_score(grid_poly, X_s5_bpfc, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

> Beta PFC: Water vs. Sucorse %15

In [21]:
# execute the nested cross-validation

start_time = time.time()
scores_s15_bpfc_linear = cross_val_score(grid_linear, X_s15_bpfc, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 105.50718975067139 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bpfc_rbf = cross_val_score(grid_rbf, X_s15_bpfc, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bpfc_poly = cross_val_score(grid_poly, X_s15_bpfc, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

> Beta NAc: Water vs. Sucrose %5 

In [22]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bnac_linear = cross_val_score(grid_linear, X_s5_bnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 121.56399178504944 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bnac_rbf = cross_val_score(grid_rbf, X_s5_bnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bnac_poly = cross_val_score(grid_poly, X_s5_bnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

> Beta NAc: Water vs. Sucrose 15%

In [23]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bnac_linear = cross_val_score(grid_linear, X_s15_bnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 101.8986964225769 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bnac_rbf = cross_val_score(grid_rbf, X_s15_bnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bnac_poly = cross_val_score(grid_poly, X_s15_bnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

> Beta vHip: Water vs. Sucrose 5%

In [24]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bhip_linear = cross_val_score(grid_linear, X_s5_bhip, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 119.3322274684906 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bhip_rbf = cross_val_score(grid_rbf, X_s5_bhip, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_bhip_poly = cross_val_score(grid_poly, X_s5_bhip, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

> Beta vHip: Water vs. Sucrose 15%

In [25]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bhip_linear = cross_val_score(grid_linear, X_s15_bhip, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 109.04044342041016 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bhip_rbf = cross_val_score(grid_rbf, X_s15_bhip, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_bhip_poly = cross_val_score(grid_poly, X_s15_bhip, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3) #poly kernel
print("--- %s seconds ---" % (time.time() - start_time))

#### Low Gamma

> Low Gamma NAc: Water vs. Sucrose 5%

In [63]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lgnac_linear = cross_val_score(grid_linear, X_s5_lgnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 14.13074803352356 seconds ---


In [65]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lgnac_rbf = cross_val_score(grid_rbf, X_s5_lgnac, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 144.84875464439392 seconds ---


In [66]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_linear = cross_val_score(grid_linear, X_s15_lgnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 15.25172758102417 seconds ---


In [67]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_rbf = cross_val_score(grid_rbf, X_s15_lgnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


[INFO] --- 151.61614227294922 seconds ---


[INFO] LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: 0.508 (0.016)


In [79]:
print('LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' %(mean(scores_s5_lgnac_linear), std(scores_s5_lgnac_linear)))

print(' LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' %(mean(scores_s5_lgnac_rbf), std(scores_s5_lgnac_rbf)))

LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: 0.508 (0.016)
 LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: 0.554 (0.010)


In [81]:
print('LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgnac_linear), std(scores_s15_lgnac_linear)))

print('LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgnac_rbf), std(scores_s15_lgnac_rbf)))


LOW GAMMA NAc, SVM w/ Linear kernel, Accuracy: 0.490 (0.010)
LOW GAMMA NAc, SVM w/ RBF kernel, Accuracy: 0.542 (0.013)


> Low Gamma NAc: Water vs. Sucrose 15%

In [27]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_linear = cross_val_score(grid_linear, X_s15_lgnac, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 116.78263092041016 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_rbf = cross_val_score(grid_rbf, X_s15_lgnac, y_s15_lgnac, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lgnac_poly = cross_val_score(grid_rbf, X_s15_lgnac, y_s15_lgnac, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


> Low Gamma vHip: Water vs. Sucrose 5%

In [28]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lghip_linear = cross_val_score(grid_linear, X_s5_lghip, y_s5_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 117.04686570167542 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lghip_rbf = cross_val_score(grid_rbf, X_s5_lghip, y_s5_lghip, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s5_lghip_poly = cross_val_score(grid_rbf, X_s5_lghip, y_s5_lghip, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


> Low Gamma vHip: Water vs. Sucrose 15%

In [29]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lghip_linear = cross_val_score(grid_linear, X_s15_lghip, y_s15_enc, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("--- %s seconds ---" % (time.time() - start_time))


--- 103.54954981803894 seconds ---


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lghip_rbf = cross_val_score(grid_rbf, X_s15_lghip, y_s15_lghip, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


In [ ]:
# execute the nested cross-validation
start_time = time.time()
scores_s15_lghip_poly = cross_val_score(grid_rbf, X_s15_lghip, y_s15_lghip, scoring='accuracy', cv=cv_outer, n_jobs=3)
print("[INFO] --- %s seconds ---" % (time.time() - start_time))


---

## Results <a class="anchor" id="results"></a>

In [37]:
# -- report performance  --

# delta 
print("\n[INFO] Results for, DELTA BLA ")
print("\n[INFO] Water vs. Sucrose 5%: ")
#('[INFO] SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_db_rbf), std(scores_s5_db_rbf)))
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_db_linear), std(scores_s5_db_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
#print('[INFO] SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_db_rbf), std(scores_s15_db_rbf)))
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_db_linear), std(scores_s15_db_linear)))
print("\n\n[INFO] Results for, DELTA NAc")
print("\n[INFO] Water vs. Sucrose 5%: ")
#print('[INFO] SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_nac_rbf), std(scores_s5_nac_rbf)))
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_dnac_linear), std(scores_s5_dnac_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
#print('[INFO] SVM w/ RBF kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_nac_rbf), std(scores_s15_nac_rbf)))
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_dnac_linear), std(scores_s15_dnac_linear)))
print('\n----')

# beta
print("\n\n[INFO] Results for, BETA PFC")
print("\n[INFO] Water vs. Sucrose 5%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_bpfc_linear), std(scores_s5_bpfc_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_bpfc_linear), std(scores_s15_bpfc_linear)))
print("\n\n[INFO] Results for, BETA NAc")
print("\n[INFO] Water vs. Sucrose 5%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_bnac_linear), std(scores_s5_bnac_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_bnac_linear), std(scores_s15_bnac_linear)))
print("\n\n[INFO] Results for, BETA vHIP ")
print("\n[INFO] Water vs. Sucrose 5%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_bhip_linear), std(scores_s5_bhip_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_bhip_linear), std(scores_s15_bhip_linear)))
print('\n----')


# Low gamma
print("\n\n[INFO] Results for, LOW GAMMA NAc")
print("\n[INFO] Water vs. Sucrose 5%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lgnac_linear), std(scores_s5_lgnac_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lgnac_linear), std(scores_s15_lgnac_linear)))
print("\n\n[INFO] Results for, LOW GAMMA vHIP ")
print("\n[INFO] Water vs. Sucrose 5%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s5_lghip_linear), std(scores_s5_lghip_linear)))
print("\n[INFO] Water vs. Sucrose 15%: ")
print('[INFO] SVM w/ Linear kernel, Accuracy: %.3f (%.3f)' % (mean(scores_s15_lghip_linear), std(scores_s15_lghip_linear)))
print('\n----')







[INFO] Results for, DELTA BLA 

[INFO] Water vs. Sucrose 5%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.578 (0.066)

[INFO] Water vs. Sucrose 15%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.598 (0.074)


[INFO] Results for, DELTA NAc

[INFO] Water vs. Sucrose 5%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.515 (0.083)

[INFO] Water vs. Sucrose 15%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.634 (0.018)

----


[INFO] Results for, BETA PFC

[INFO] Water vs. Sucrose 5%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.562 (0.075)

[INFO] Water vs. Sucrose 15%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.629 (0.077)


[INFO] Results for, BETA NAc

[INFO] Water vs. Sucrose 5%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.533 (0.114)

[INFO] Water vs. Sucrose 15%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.633 (0.016)


[INFO] Results for, BETA vHIP 

[INFO] Water vs. Sucrose 5%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.494 (0.057)

[INFO] Water vs. Sucrose 15%: 
[INFO] SVM w/ Linear kernel, Accuracy: 0.63

---

---

## 